In [1]:
import os
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)
import seaborn as sns
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split

C:\ProgramData\Anaconda3\envs\min\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
train = pd.read_csv("LSTM_DATA.csv", engine="python", encoding="utf-8", index_col=0)
train["date_time"] = train["date_time"].astype("datetime64")
train["Month"] = train["date_time"].dt.month
train.drop(columns="date_time",axis=1,inplace=True)
train

,num,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유,hour,day,건물용도,Month
0,1,8179.056,17.6,2.5,92.0,0.8,0.0,0,0,0,0,3,6
1,1,8135.640,17.7,2.9,91.0,0.3,0.0,0,0,1,0,3,6
2,1,8107.128,17.5,3.2,91.0,0.0,0.0,0,0,2,0,3,6
3,1,8048.808,17.1,3.2,91.0,0.0,0.0,0,0,3,0,3,6
4,1,8043.624,17.0,3.3,92.0,0.0,0.0,0,0,4,0,3,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,60,4114.368,27.8,2.3,68.0,0.0,0.7,1,1,19,0,2,8
122396,60,3975.696,27.3,1.2,71.0,0.0,0.0,1,1,20,0,2,8
122397,60,3572.208,27.3,1.8,71.0,0.0,0.0,1,1,21,0,2,8
122398,60,3299.184,27.1,1.8,74.0,0.0,0.0,1,1,22,0,2,8


In [3]:
def Data_cleaning (data, num, plot=False) :
    data = data[data['num']==num]
    nc_list = data.corr()[data.corr()["전력사용량(kWh)"].isna() == True].index.values
    data.drop(columns=nc_list, axis=1, inplace=True)
    x = np.arange(0,2040,1)
    if plot :
        plt.figure(figsize=(16, 9))
        for i in np.arange(0,6,1) :
            plt.subplot(6,1,i+1)
            sns.lineplot(y=data.iloc[:,i],x=x)
            plt.xlabel('Month')
            plt.xticks(np.arange(0,2040,24*30.5), ['6월', '7월', '8월'],rotation=20)
            plt.ylabel(data.columns[i])
        plt.show()
    return data

In [26]:
def Data_splitting (data, data_shape=False, test_size=0.2, plot=False, parameter = True) :
    x_train, x_test, y_train, y_test = train_test_split(train_num.drop('전력사용량(kWh)', 1), 
                                                    train_num['전력사용량(kWh)'], test_size=test_size,
                                                        random_state=0, shuffle=False)
    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean) / x_train_std
    x_test = (x_test - x_train_mean) / x_train_std
    y_train_mean = y_train.mean()
    y_train_std = y_train.std()
    y_train = (y_train - y_train_mean) / y_train_std
    y_test = (y_test - y_train_mean) / y_train_std
    
    if data_shape :
        print("X_train 데이터 shape :", x_train.shape)
        print("X_test 데이터 shape :", x_test.shape)
        print("Y_train 데이터 shape :", y_train.shape)
        print("Y_test 데이터 shape :", y_test.shape)
    if plot :
        normalized_train = x_train.melt(var_name='Column', value_name='Normalized')
        plt.figure(figsize=(12, 6))
        ax = sns.violinplot(x='Column', y='Normalized', data=normalized_train)
        _ = ax.set_xticklabels(x_train.keys(), rotation=90)
        plt.show()
    if parameter :
        return x_train, x_test, y_train, y_test, x_train_mean, x_train_std
    else :
        return x_train, x_test, y_train, y_test

In [27]:
def windowed_dataset(x,y, window_size, batch_size, shuffle):
    # Dataset으로 변환
    ds_x = tf.data.Dataset.from_tensor_slices(x)
    # x일의 데이트를 통해 -> x+1일을 예측 shirh=1일, stride = 간격, drop_remainder = 끝 맞추기
    ds_x = ds_x.window(window_size, shift=1, drop_remainder=True)
    ds_x = ds_x.flat_map(lambda x : x.batch(window_size))
    ds_y = tf.data.Dataset.from_tensor_slices(y[window_size-1:])
    ds = tf.data.Dataset.zip((ds_x,ds_y))
    if shuffle:
        ds = ds.shuffle(1000)
    #                  미리 배치 하는 갯수 
    return ds.batch(batch_size).prefetch(1)

In [37]:
mission = pd.read_csv("mission.csv", engine="python", encoding="utf-8", index_col=0)
mission.rename(columns={"강수량(mm, 6시간)" :"강수량(mm)", 
                        "일조(hr, 3시간)" :"일조(hr)"}, inplace=True)

In [39]:
train_num = Data_cleaning(train, 25)
x_train, x_test, y_train, y_test, mean, std = Data_splitting(train_num)
mission_data = Mission_data(mission, 25, mean=mean, std=std, x_test=x_test)
train_data = windowed_dataset(x_train, y_train, window_size=8, batch_size=15, shuffle=True)
test_data = windowed_dataset(x_test, y_test, window_size=8, batch_size=15, shuffle=False)

In [ ]:
def mission (model_name, optimizer, loss, dropout, recurrent_dropout) :
    model_result = []
    for i in train["num"].unique() :
        train_num = Data_cleaning(train, i)
        x_train, x_test, y_train, y_test = Data_splitting(train_num)
        mission_data = Mission_data(mission, 25, mean=mean, std=std, x_test=x_test)
        train_data = windowed_dataset(x_train, y_train, window_size=8, batch_size=15, shuffle=True)
        test_data = windowed_dataset(x_test, y_test, window_size=8, batch_size=15, shuffle=False)
        print(f"--------------------------------{i}번째 항목")
        model, pred, score, history = Data_Modeling(train_data, test_data, model_name=model_name,
                                                    dropout=dropout,recurrent_dropout=recurrent_dropout, optimizer=optimizer,loss=loss)
        score_mission = model.predict(mission_data)
        model_result.append(score_mission)
    

    return model_result